In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import csv

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense , Activation, Dropout,BatchNormalization,Input
from tensorflow.keras.optimizers import Adam ,RMSprop
from tensorflow.keras.models import Model
from tensorflow.keras import  backend as K
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping
from tensorflow.keras.backend import clear_session

from sklearn.model_selection import train_test_split, ParameterGrid, ParameterSampler
from sklearn.metrics import mean_absolute_error
from random import random,randrange
from operator import itemgetter
import timeit

from FCMnR import FCMnR_model


In [2]:
# Thanks to https://www.kaggle.com/code/julienjta/flight-price-prediction-98-47-r2-score
def preprocessing(df):
    #Encode the ordinal variables "stops" and "class".
    df["stops"] = df["stops"].replace({'zero':0,'one':1,'two_or_more':2}).astype(int)
    df["class"] = df["class"].replace({'Economy':0,'Business':1}).astype(int)
    
    #Create the dummy variables for the cities, the times and the airlines.
    dummies_variables = ["airline","source_city","destination_city","departure_time","arrival_time"]
    dummies = pd.get_dummies(df[dummies_variables], drop_first= True)
    df = pd.concat([df,dummies],axis=1)
    
    #Create the dummy variables for the cities, the times and the airlines.
    df = df.drop(["flight","airline","source_city","destination_city","departure_time","arrival_time"],axis=1)
    
    return df

In [3]:
def load_data():
    df = pd.read_csv("Clean_Dataset.csv",index_col=0)

    df = preprocessing(df)
    print("There are {} observations for {} predictors.".format(df.shape[0],df.shape[1]))
    df.head()    
    X = df.copy()
    y = X.pop("price")
    xtrain,xtest,ytrain,ytest = train_test_split(X,y,random_state = 1,test_size=0.2, shuffle=True)
    xtrain,xvalid,ytrain,yvalid = train_test_split(xtrain,ytrain,random_state = 1,test_size=0.2, shuffle=True)
    return xtrain,xtest,xvalid,yvalid,ytrain,ytest


In [4]:
xtrain,xtest,xvalid,yvalid,ytrain,ytest=load_data()
#DATA/TASK INFORMATION:
architecture_name="fcmnr"
problem_type="prediction"
num_features=xtrain.shape[1]
input_shape =(xtrain.shape[1])
training_and_validation_samples=len(ytrain)+len(yvalid)
xtrain.head()

There are 300153 observations for 30 predictors.


,stops,class,duration,days_left,airline_Air_India,airline_GO_FIRST,airline_Indigo,airline_SpiceJet,airline_Vistara,source_city_Chennai,...,departure_time_Early_Morning,departure_time_Evening,departure_time_Late_Night,departure_time_Morning,departure_time_Night,arrival_time_Early_Morning,arrival_time_Evening,arrival_time_Late_Night,arrival_time_Morning,arrival_time_Night
47870,1,0,10.17,26,0,0,0,0,1,0,...,0,0,0,1,0,0,1,0,0,0
82547,1,0,4.92,39,0,0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0
112828,1,0,12.92,28,1,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1
125170,0,0,2.42,36,0,0,0,0,1,0,...,1,0,0,0,0,0,0,0,1,0
100057,1,0,4.42,35,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,0,1


In [5]:
def evaluate_fitness(input_shape,n_layers,activation_function,learning_rate,batch_size,hp_dataset_name,weights_name,max_epochs,patience_epochs):
    clear_session()
    #CREATE MODEL
    model=FCMnR_model(n_layers,input_shape,activation_function,learning_rate)
    start_time = timeit.default_timer()
    history = model.fit(xtrain,ytrain,
                        batch_size=batch_size,
                        epochs=max_epochs,
                        callbacks=[EarlyStopping(patience=patience_epochs)],validation_data=(xvalid,yvalid))
    end_time = timeit.default_timer()
    
    #EVALUATE MODEL
    prediction=model.predict(xtest)
    mae_test=mean_absolute_error(ytest,prediction)


    #SAVE THE WEIGHTS
    model.save(weights_name+".h5")

    #SAVE THE HYPERPARAMS AND THE METRIC
    with open(hp_dataset_name, mode='a+') as hp_dataset:
        hp_dataset_writer=csv.writer(hp_dataset,delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        hp_dataset_writer.writerow([architecture_name,
                                problem_type,
                                num_features,
                                training_and_validation_samples,
                                n_layers,
                                input_shape,
                                activation_function,
                                learning_rate,
                                batch_size,
                                str(len(history.history['loss'])),
                                end_time-start_time,
                                mae_test
                                ])
    return mae_test


In [8]:
def random_gridsearch(population_size,input_shape,n_layers,activation_function,learning_rate,batch_size,hp_dataset_name,max_epochs,patience_epochs):
        dict_all_hyperparams=dict(n_layers=n_layers,
                                learning_rate=learning_rate,
                                activation_function=activation_function,
                                batch_size=batch_size,
                                )
        r_grid_search_population=list(ParameterSampler(dict_all_hyperparams,population_size))
        
        RGS_evaluated_hparams=[]
        with open("Logs_RandomGridSearch.csv", mode='a+') as logs_dataset:
                logs_dataset_writer=csv.writer(logs_dataset,delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
                logs_dataset_writer.writerow(["population: "+str(population_size)])
                logs_dataset_writer.writerows(dict(x=r_grid_search_population).values())
        print(r_grid_search_population)

        
        for i in range(len(r_grid_search_population)):
                weights_name='{}-{}-{}-{}'.format(r_grid_search_population[i]['n_layers'],r_grid_search_population[i]['activation_function'],r_grid_search_population[i]['learning_rate'],r_grid_search_population[i]['batch_size'])
                metric=evaluate_fitness(input_shape,
                                r_grid_search_population[i]['n_layers'],
                                r_grid_search_population[i]['activation_function'],
                                r_grid_search_population[i]['learning_rate'],
                                r_grid_search_population[i]['batch_size'],
                                hp_dataset_name,
                                weights_name,
                                max_epochs,
                                patience_epochs
                                )
                
                with open("Logs_RandomGridSearch.csv", mode='a+') as logs_dataset:
                        logs_dataset_writer=csv.writer(logs_dataset,delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
                        logs_dataset_writer.writerow(["i:"+str(i)+"Metric:"+str(metric)])
                print("i",i,"Mae:",metric)

                RGS_evaluated_hparams.insert(len(RGS_evaluated_hparams),{"hparam":i,"metric":metric})
        rgs_top_hparam=sorted(RGS_evaluated_hparams,key=itemgetter('metric'),reverse=True)[0]['hparam']
        
        with open("Logs_RandomGridSearch.csv", mode='a+') as logs_dataset:
                        logs_dataset_writer=csv.writer(logs_dataset,delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
                        logs_dataset_writer.writerow("END")
                        logs_dataset_writer.writerows(sorted(RGS_evaluated_hparams,key=itemgetter('metric'),reverse=True)[0]['metric'],r_grid_search_population[rgs_top_hparam])
        
        return sorted(RGS_evaluated_hparams,key=itemgetter('metric'),reverse=True)[0]['metric'],r_grid_search_population[rgs_top_hparam]

In [9]:

#HYPERPARAMETERS DEFINITION
n_layers = [1,2,3]
activation_function=['relu','tanh','sigmoid','elu']
learning_rate=[0.01,0.001,0.0001,0.00001]
batch_size=[16,32,64,128]
max_epochs=1
patience_epochs=20

#FILES NAME
hp_dataset_name="hyperparams_with_metric.csv"

#ALGORITHM PARAMS
population_size=5

random_gridsearch(population_size,input_shape,n_layers,activation_function,learning_rate,batch_size,hp_dataset_name,max_epochs,patience_epochs)








[{'n_layers': 1, 'learning_rate': 0.0001, 'batch_size': 128, 'activation_function': 'tanh'}, {'n_layers': 1, 'learning_rate': 1e-05, 'batch_size': 64, 'activation_function': 'relu'}, {'n_layers': 3, 'learning_rate': 0.01, 'batch_size': 16, 'activation_function': 'relu'}, {'n_layers': 2, 'learning_rate': 1e-05, 'batch_size': 32, 'activation_function': 'tanh'}, {'n_layers': 3, 'learning_rate': 1e-05, 'batch_size': 16, 'activation_function': 'relu'}]
Train on 192097 samples, validate on 48025 samples
192097/192097 [==============================] - 30s 154us/sample - loss: 20884.1844 - mean_absolute_error: 20884.1953 - val_loss: 20989.8490 - val_mean_absolute_error: 20989.8535
i 0 Mae: 20796.95416327683
Train on 192097 samples, validate on 48025 samples
192097/192097 [==============================] - 56s 293us/sample - loss: 20784.4758 - mean_absolute_error: 20784.4883 - val_loss: 20806.3210 - val_mean_absolute_error: 20806.3086
i 1 Mae: 20613.38234510436
Train on 192097 samples, validat

KeyboardInterrupt: 